# Q46 아기 상어

In [ ]:
# 나의 답안

from collections import deque
# import sys
# input = lambda: sys.stdin.readline().rstrip()

n = int(input())
grid = [[0] * n for _ in range(n)]
size = 2
quota = 0

# 입력
for a in range(n):
    line = list(map(int, input().split()))
    for b in range(n):
        grid[a][b] = line[b]
        if line[b] == 9:
            x, y = a, b
            grid[a][b] = 0

# 몇 초?
answer = 0

# 아기 상어의 이동
def bfs(x, y):
    global size
    global quota
    global answer

    time = [[-1] * n for _ in range(n)]
    queue = deque([(x, y)])
    time[x][y] = 0

    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]
    result = []
    limit = int(1e9)

    while queue:
        cx, cy = queue.popleft()
        if 1 <= grid[cx][cy] < size and time[cx][cy] <= limit:
            # 최단 거리에 존재하는 물고기 다 추가
            limit = time[cx][cy]
            result.append((cx, cy))
            # print(result)

        for i in range(4):
            nx, ny = cx + dx[i], cy + dy[i]
            # 이동할 수 있는 경우
            if 0 <= nx < n and 0 <= ny < n:
                if 0 <= grid[nx][ny] <= size and time[nx][ny] == -1:
                    time[nx][ny] = time[cx][cy] + 1
                    if time[nx][ny] <= limit:
                        queue.append((nx, ny))

    # 먹을 수 있는 물고기가 공간에 없음
    if not result:
        return False

    # 먹을 수 있는 물고기가 공간에 있음
    else:
        result.sort()
        cx, cy = result[0]
        quota += 1
        answer += time[cx][cy]
        # print(f"EAT ({cx}, {cy}), fish {grid[cx][cy]}, quota {quota} / {size}, answer {answer}")
        grid[cx][cy] = 0

        # 자신의 크기와 같은 만큼 물고기를 먹음
        if quota >= size:
            quota = 0
            size += 1

        return cx, cy

while True:
    coord = bfs(x, y)
    if coord == False:
        break
    else:
        x, y = coord

print(answer)

* BFS를 이용하여 최단 거리를 구하자.
* 세부 조건이 많아 실수하기 쉽다.
* BFS 자체의 시간 복잡도가 그렇게 높지 않고 $2 \leq N \leq 20$ 이기도 해서 그냥 쫙 돌리고 안에서 최소거리를 찾는 게 속편하다.
* global 많이 나올 것 같으면 함수를 쪼개자.

In [2]:
from collections import deque
# import sys
# input = lambda: sys.stdin.readline().rstrip()

n = int(input())
grid = [[0] * n for _ in range(n)]
size = 2
dx = [-1, 0, 1, 0]
dy = [0, -1, 0, 1]

# 입력
for a in range(n):
    line = list(map(int, input().split()))
    for b in range(n):
        grid[a][b] = line[b]
        if line[b] == 9:
            x, y = a, b
            grid[a][b] = 0


# 아기 상어의 이동
def bfs(x, y):
    time = [[-1] * n for _ in range(n)]
    queue = deque([(x, y)])
    time[x][y] = 0

    while queue:
        x, y = queue.popleft()

        for i in range(4):
            nx, ny = x + dx[i], y + dy[i]
            # 이동할 수 있는 경우
            if 0 <= nx < n and 0 <= ny < n:
                if 0 <= grid[nx][ny] <= size and time[nx][ny] == -1:
                    time[nx][ny] = time[x][y] + 1
                    queue.append((nx, ny))

    return time

# 물고기 찾기
def find(time):
    x, y = 0, 0
    min_time = int(1e9)

    for i in range(n):
        for j in range(n):
            if time[i][j] != -1 and 1 <= grid[i][j] < size:
                if time[i][j] < min_time:
                    x, y = i, j
                    min_time = time[i][j]

    # 먹을 수 있는 물고기가 공간에 없음
    if min_time >= int(1e9):
        return None
    # 먹을 수 있는 물고기가 공간에 있음
    else:
        return x, y, min_time

answer = 0 # 최종답안
quota = 0 # 현재크기에서 먹은 양

while True:
    coord = find(bfs(x, y))
    if coord == None:
        print(answer)
        break
    else:
        x, y = coord[0], coord[1]
        answer += coord[2]
        grid[x][y] = 0
        quota += 1

        if quota >= size:
            size += 1
            quota = 0

4
4 3 2 1
0 0 0 0
0 0 9 0
1 2 3 4
14


# Q47 청소년 상어

In [49]:
from copy import deepcopy

grid = [[0] * 4 for _ in range(4)]
dirs = dict()
locs = dict()

dx = [0, -1, -1, 0, 1, 1, 1, 0, -1]
dy = [0, 0, -1, -1, -1, 0, 1, 1, 1]

for x in range(4):
    line = list(map(int, input().split()))
    for y in range(4):
        grid[x][y] = line[2 * y]
        dirs[line[2 * y]] = line[2 * y + 1]
        locs[line[2 * y]] = (x, y)

def move_fish(i):
    # i: 물고기 번호
    x, y = locs[i]

    # 이동 가능여부 체크
    # 이동 불가능하면 45도 반시계 회전
    for _ in range(8):
        nx, ny = x + dx[dirs[i]], y + dy[dirs[i]]
        if 0 <= nx < 4 and 0 <= ny < 4 and grid[nx][ny] != "S":
            if grid[nx][ny] != -1:
                shift = grid[nx][ny]
                locs[i], locs[shift] = locs[shift], locs[i]
                grid[x][y], grid[nx][ny] = grid[nx][ny], grid[x][y]

            else:
                locs[i] = (nx, ny)
                grid[nx][ny] = i
                grid[x][y] = -1
            return

        else:
            dirs[i] += 1
            if dirs[i] >= 9:
                dirs[i] = 1
    return

def move_shark(sx, sy, sd):
    nx, ny = sx, sy
    avail_prey = []
    while True:
        nx += dx[sd]
        ny += dy[sd]
        if nx < 0 or 4 <= nx or ny < 0 or 4 <= ny:
            break
        if grid[nx][ny] != -1:
            prey = grid[nx][ny]
            avail_prey.append((prey, nx, ny, dirs[prey]))

    return avail_prey

answer = 0

def movement(grid, sx, sy, sd, eaten):
    grid[sx][sy] = "S"

    for i in range(1, 17):
        if i not in eaten:
            move_fish(i)
    # 상어가 이동한다
    avail_prey = move_shark(sx, sy, sd)

    if not avail_prey:
        return 0
    else:
        results = []
        grid[sx][sy] = -1
        for prey, px, py, pd in avail_prey:
            print(prey, px, py, pd, eaten)
            eaten.append(prey)
            new_grid = deepcopy(grid)
            results.append(prey + movement(new_grid, px, py, pd, eaten))
        return max(results)

# 상어는 (0, 0)에 들어간다
# 상어의 방향은 (0, 0)에 있었던 물고기 방향과 같다
prey = grid[0][0]
print(prey + movement(grid, 0, 0, dirs[prey], [prey]))

In [38]:
grid.copy()

[['S', 14, 1, 'S'], [8, 13, 16, -1], [2, -1, 4, 11], [-1, -1, 9, 5]]

보류하는 게 좋아 보인다

# Q48 어른 상어